# Experiment 2
Need to run experiment 1 first

In [1]:
import pathlib
import functools
import pandas as pd
import numpy as np
import math
import pickle

from tqdm.auto import tqdm

from videoannotator import data, dp, experiments

## helpers

In [2]:
dps = {
    'eps0.1': functools.partial(dp.EpsilonGreedyMean, epsilon=0.1),
    'eps0.25': functools.partial(dp.EpsilonGreedyMean, epsilon=0.25),
    'eps0.5': functools.partial(dp.EpsilonGreedyMean, epsilon=0.5),
    'rr': dp.RoundRobin,
    'greedy': dp.GreedyOracle,
    'ucb': dp.UCBMean,
    'ucb.1': functools.partial(dp.UCBMean, c=1e-1),
    # 'ecbwema': dp.UCBEWMA,
    # 'eps0.25ewma': functools.partial(dp.EpsilonGreedyEWMA, epsilon=0.25),
}

@functools.lru_cache(10_000)
def run(dp_name, d, label):
    dp = dps[dp_name]
    try:
        return dict(
            dp=dp,
            d=d,
            label=label,
            res=dp(label=label, d=d, _seed_base=1).run(n=1_000 // d),
        )
    except Exception as e:
        raise ValueError(f'Error in dp={dp}, d={d}, label={label}: {e}')
        
def get_best(r):
    res = r['res']
    return max(
        (
            (res.average_precision.mean, res.average_precision)
            for _, res in res.res_hist
        ),
        key=lambda x: x[0],
    )[1]

def get_algo_detes(r):
    dp = r['dp']
    if dp.__class__ == functools.partial:
        return dict(
            algo=dp.func.__name__,
            kws=dp.keywords,
            args=dp.args,
        )
    else:
        return dict(algo=dp.__name__)
    
def eps(q):
    if pd.notnull(q) and 'epsilon' in q:
        return q.get('epsilon') == .25
        
    return True

## run experiment 2

In [3]:
# this takes ~20 mins on a machine with 64 CPUs and 64GB RAM
path_results_pkl = 'exp2_results.pkl'
if not pathlib.Path(path_results_pkl).exists():
    # takes a few hours
    items = [
        (dp_name, d, label)
        for i, dp_name in enumerate(dps)
        for d in (25,)
        for label in sorted(data.cfg.LABELS)
    ]
    res = [run(*q) for q in tqdm(items)]
    pickle.dump(res, open(path_results_pkl, 'wb'))
else:
    res = pickle.load(open(path_results_pkl, 'rb'))

## analyze

In [4]:
# this object is computed in experiment 1
ap_va = pickle.load(open('ap_va_best.pkl', 'rb'))
ap_va_best = {k: v['best'] for k, v in ap_va.items()}
ap_va_last = {k: v['last'] for k, v in ap_va.items()}

df_res = pd.DataFrame(
    dict(
        d=r['d'],
        label=r['label'],
        best_idx=r['res'].best_idx,
        best_metric=get_best(r),
        best_metric_mean=get_best(r).mean,
        last_metric=r['res'].last_metric,
        last_metric_mean=r['res'].last_metric.mean,
        res=r['res'],
        after_5=r['res'].res_hist[5][1].average_precision.mean,
        after_10=r['res'].res_hist[10][1].average_precision.mean,
        **get_algo_detes(r),
    )
    for r in res
)
df_res = df_res.assign(
    ap_va_best=df_res.label.map(ap_va_best),
    ap_va_last=df_res.label.map(ap_va_last),
)
d = df_res[df_res.algo == 'UCBMean'].groupby(['d', 'label']).apply(lambda x: pd.Series(dict(
    best_va=x.ap_va_best.max(),
    best_bandit=x.best_metric_mean.max(),
    last_va=x.ap_va_last.max(),
    last_bandit=x.last_metric_mean.max(),
)))
d = d.assign(
    lift_best=d.best_bandit - d.best_va,
    lift_max=d.last_bandit - d.last_va,
)
dfr = df_res[df_res.kws.apply(eps)]
e = dfr[dfr.d == 25].groupby(['algo', 'label']).apply(lambda x: pd.Series(dict(
    best_va=x.ap_va_best.max(),
    best_bandit=x.best_metric_mean.max(),
)))
e = e.assign(lift_best=e.best_bandit - e.best_va,)
algos = ['RoundRobin', 'GreedyOracle', 'EpsilonGreedyMean', 'UCBMean']
cols = ['min', '25%', '50%', '75%', 'max']

## table 3

In [5]:
ee = e.reset_index().groupby('algo').lift_best.agg(
    p10=functools.partial(np.percentile, q=10),
    p25=functools.partial(np.percentile, q=25),
    p50=functools.partial(np.percentile, q=50),
    p75=functools.partial(np.percentile, q=75),
    p90=functools.partial(np.percentile, q=90)
).loc[algos]
(ee * 100).round(1)

,p10,p25,p50,p75,p90
algo,,,,,
RoundRobin,-5.9,-0.3,2.5,7.4,15.1
GreedyOracle,-5.4,0.9,3.9,9.6,16.2
EpsilonGreedyMean,-6.5,-0.6,2.9,7.6,16.7
UCBMean,-7.5,-0.2,3.4,9.2,15.9


## cumulative results
Add up the gain from experiment 1 to this gain and compute the median across labels.

In [6]:
e1 = pickle.load(open('exp1_gain.pkl', 'rb'))
f = e.reset_index()
e2 = f[f.algo == 'UCBMean'].set_index('label').lift_best.to_dict()
comb = pd.Series({
    k: e1[k] + e2[k]
    for k in e1
    if pd.notnull(e1[k]) and pd.notnull(e2[k])
})
(np.percentile(comb, q=50) * 100).round(1)

8.3